In [77]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample
from sklearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE  # For resampling if necessary
import plotly.express as px
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
import pandas as pd
warnings.filterwarnings('ignore')

pd.options.mode.chained_assignment = None  # default='warn'

In [101]:
#Merge Gameplay review together
g_set  = pd.read_csv('datasets/__sprint2_filtered_gameplay_review_set_with_sentiment_scores_standardized.csv')
u_set  = pd.read_csv('datasets/__sprint2_filtered_usability_review_set_with_sentiment_scores_standardized.csv')
f_set  = pd.read_csv('datasets/__sprint2_filtered_functionality_review_set_with_sentiment_scores_standardized.csv')

g_set.drop(columns=['Unnamed: 0'], inplace=True)
u_set.drop(columns=['Unnamed: 0'], inplace=True)
f_set.drop(columns=['Unnamed: 0'], inplace=True)


In [102]:
f_set.head()

,recommendationid,clean_sentence,tokenized_sentence,voted_up,order,informative,heuristic,RoBERTa-Twitter_sentiment,RoBERTa-Twitter_score,VADER_sentiment,VADER_score,Sarcasm_Prob,RoBERTa-Twitter_score_standardized,VADER_score_standardized
0,24845437,issues running game getting fps options turn c...,ok for those having issues running the game an...,1,3,1.0,"[('F',)]",Neutral,0.737113,Positive,0.2960,0.121454,0.224146,0.302693
1,24845437,also turn vsync want higher fps,also turn off vsync if you want higher than 60...,1,5,1.0,"[('F',)]",Neutral,0.732211,Positive,0.0772,0.232653,0.209113,0.082406
2,24845437,game runs really smooth,after i did that the game runs really smooth f...,1,6,1.0,"[('F',)]",Positive,0.866402,Neutral,0.0000,0.858768,0.620631,0.004682
3,24845437,many ppl rating negative havent discovered opt...,so many ppl are rating it negative just becaus...,1,8,1.0,"[('F',)]",Negative,0.813215,Negative,-0.1280,0.675439,0.457524,-0.124188
4,24845437,nothing worse weapon running juice,there is nothing worse than any weapon running...,1,44,1.0,"[('F',)]",Negative,0.922386,Positive,0.0910,0.345169,0.792314,0.096300


In [105]:
print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

24.Mar 2025 14:35:58


In [107]:
g_ids = g_set['recommendationid'].unique()
u_ids = u_set['recommendationid'].unique()
f_ids = f_set['recommendationid'].unique()

df2 = pd.DataFrame(data=None, columns=g_set.columns, index=g_set.index)
#df2.dropna(inplace=True)
#print(df2.head())

data_map = [
    u_set,
    g_set,
    f_set
]

data_ids=[u_ids, g_ids, f_ids]

counter = 0

names = ['usability', 'gameplay', 'functionality']

output = [
    pd.DataFrame(data=None, columns=u_set.columns, index=u_set.index),
    pd.DataFrame(data=None, columns=g_set.columns, index=g_set.index),
    pd.DataFrame(data=None, columns=f_set.columns, index=f_set.index)
]

for i in range(len(data_map)):
    print('Merging ' + names[counter] + ' data')
    output_df = output[counter]
    
    for k in data_ids[i]:
        wip_data = data_map[i].loc[data_map[i]['recommendationid'] == k]
        wip_data.sort_values('order',ascending=True, inplace=True)
        average_row = wip_data.select_dtypes(include='number').mean()
        merged_clean_sentence = '. '.join(wip_data['clean_sentence'])
        merged_tokenized_sentence = '. '.join(wip_data['tokenized_sentence'])
        average_row['clean_sentence'] = merged_clean_sentence
        average_row['tokenized_sentence'] = merged_tokenized_sentence
        average_row['heuristic'] = wip_data['heuristic'].unique()
        average_row['RoBERTa-Twitter_sentiment'] ='TBD'
        average_row['VADER_sentiment'] ='TBD'
        average_row = average_row[data_map[i].columns]
        average_row['sentence_count'] = len(data_map[i].loc[data_map[i]['recommendationid'] == k])
        output_df = pd.concat([output_df, pd.DataFrame([average_row])], ignore_index=True)
        output[counter] = output_df
    counter += 1
    if counter == 1:
        print(output_df.tail(20))
        #break


Merging usability data
       recommendationid                                     clean_sentence  \
23090        70211748.0  youre using joy stick controller etc set ship ...   
23091        70219907.0  immediately completing tutorial missions reali...   
23092        70227141.0  best game ive played great story lots options ...   
23093        70263361.0  wouldnt first game hold hand tightly enough do...   
23094        70265878.0  theres bounties appear pirates threaten existe...   
23095        70270920.0  thing wouldve liked see control ship manual do...   
23096        70275685.0                                  best played mates   
23097        70283350.0  game designed play mouse keyboard. consistency...   
23098        70294281.0  good game little bit glitchy got used multplay...   
23099        70302014.0                      story bit hard follow amazing   
23100        70304870.0                  best redeem story gaming industry   
23101        70306978.0          easy get

In [108]:
print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

24.Mar 2025 14:52:40


In [132]:
output[0].dropna(inplace=True)
output[1].dropna(inplace=True)
output[2].dropna(inplace=True)

merged_df = pd.concat([output[0], output[1], output[2]], ignore_index=True)

print(len(output[0]))
print(len(output[1]))
print(len(output[2]))
print(len(merged_df))

merged_df['tokenized_sentence_no_punct'] = merged_df['tokenized_sentence'].str.replace('.', '', regex=False)
merged_df['clean_sentence_no_punct'] = merged_df['clean_sentence'].str.replace('.', '', regex=False)

output[0].to_csv('datasets/__sprint2_usability_review_sentences_remerged.csv', index=False)
output[1].to_csv('datasets/__sprint2_gameplay_review_sentences_remerged.csv', index=False)
output[2].to_csv('datasets/__sprint2_functionality_review_sentences_remerged.csv', index=False)
merged_df.to_csv('datasets/__sprint2_All_heuristics_review_sentences_remerged.csv', index=False)

9304
38364
20563
68231


In [134]:


df  = pd.read_csv('datasets/steam_reviews_constructiveness_1.5k.csv')

rev_df = pd.read_csv('datasets/__sprint2_All_heuristics_review_sentences_remerged.csv')
print(len(rev_df))
rev_df.dropna(subset=['tokenized_sentence'], how='all', inplace=True)
print(len(rev_df))
# Example stopwords list, modify as needed
stop_words_list = set(STOPWORDS)

# Generate frequency counts from the 'business_category' column
#counts = Counter(df["game"].dropna().apply(lambda x: str(x)))

#1 = Constructive
#0 = not constructive

#def vect_funt(review):
#    texts2 = review
#    return vectorizer.transform(review)

#df['vectorised_reviews'] = df.apply(lambda x: vect_funt([x['review']]), axis=1)

df.head()




68231
68231


,id,game,review,author_playtime_at_review,voted_up,votes_up,votes_funny,constructive
0,1,Among Us,This game can suck my balls before I play it a...,6,False,1,0,0
1,2,Among Us,Very fun little party game! Even better with f...,11,True,0,0,1
2,3,Among Us,if you're lonely don't bother but if you're no...,40,True,2,1,0
3,4,Among Us,fun and anoyying,80,True,0,0,0
4,5,Among Us,when impostor is sus...,51,True,0,0,0


In [136]:
print(df['constructive'].value_counts())
weight_0 = 1
weight_1 = len(df[df['constructive'] == 0]['constructive']) / len(df[df['constructive'] == 1]['constructive'])

print(weight_1)

constructive
0    921
1    540
Name: count, dtype: int64
1.7055555555555555


In [138]:
rev_df.head(10)

,recommendationid,clean_sentence,tokenized_sentence,voted_up,order,informative,heuristic,RoBERTa-Twitter_sentiment,RoBERTa-Twitter_score,VADER_sentiment,VADER_score,Sarcasm_Prob,RoBERTa-Twitter_score_standardized,VADER_score_standardized,sentence_count,tokenized_sentence_no_punct,clean_sentence_no_punct
0,24845437.0,press craft item inventory menu took like minu...,press e to craft an item while in the inventor...,1.0,24.5,1.0,"[""[('U',)]""]",TBD,0.727397,TBD,0.23165,0.287373,0.197922,0.236190,2.0,press e to craft an item while in the inventor...,press craft item inventory menu took like minu...
1,24845526.0,keep pressing mouse circle fills,what this with keep pressing mouse down until ...,0.0,1.0,1.0,"[""[('U',)]""]",TBD,0.626157,TBD,0.00000,0.133741,-0.111161,0.002472,1.0,what this with keep pressing mouse down until ...,keep pressing mouse circle fills
2,24845539.0,cons horrible proformance highend simple times...,cons horrible proformance on highend pc to sim...,0.0,6.5,1.0,"[""[('U',)]""]",TBD,0.928605,TBD,-0.80275,0.078206,0.812205,-0.807446,2.0,cons horrible proformance on highend pc to sim...,cons horrible proformance highend simple times...
3,24845671.0,preorder bonus ship gives extra inventory slot...,the preorder bonus ship only gives you 1 extra...,1.0,4.0,1.0,"[""[('U',)]""]",TBD,0.852708,TBD,0.77830,0.896714,0.580493,0.787721,1.0,the preorder bonus ship only gives you 1 extra...,preorder bonus ship gives extra inventory slot...
4,24845726.0,runs like shit crappy console port,runs like shit and its just a crappy console port,0.0,0.0,1.0,"[""[('U',)]""]",TBD,0.942896,TBD,-0.69080,0.687808,0.855833,-0.694496,1.0,runs like shit and its just a crappy console port,runs like shit crappy console port
5,24845880.0,1110 would molest virgin planet,1110 would molest a virgin planet again,1.0,0.0,1.0,"[""[('U',)]""]",TBD,0.791999,TBD,-0.47670,0.319447,0.395149,-0.478485,1.0,1110 would molest a virgin planet again,1110 would molest virgin planet
6,24845905.0,pvp aspect awful half naked man run hill demol...,the pvp aspect is awful as a half naked man ca...,0.0,4.0,1.0,"[""[('U',)]""]",TBD,0.929842,TBD,0.05160,0.504178,0.815982,0.054533,1.0,the pvp aspect is awful as a half naked man ca...,pvp aspect awful half naked man run hill demol...
7,24845989.0,changing mouse sensitivity unresponsive,changing mouse sensitivity is unresponsive,0.0,1.0,1.0,"[""[('U',)]""]",TBD,0.725801,TBD,0.00000,0.294270,0.193051,0.002472,1.0,changing mouse sensitivity is unresponsive,changing mouse sensitivity unresponsive
8,24846075.0,well far terrible experince. could use help,well so far a terrible experince. could use so...,0.0,1.5,1.0,"[""[('U',)]""]",TBD,0.695640,TBD,0.04425,0.400029,0.100970,0.047117,2.0,well so far a terrible experince could use som...,well far terrible experince could use help
9,24846136.0,cons poorly optimized action takes longer hold...,cons very poorly optimized doing any action ta...,0.0,0.0,1.0,"[""[('U',)]""]",TBD,0.914853,TBD,0.14770,0.120171,0.770221,0.151491,1.0,cons very poorly optimized doing any action ta...,cons poorly optimized action takes longer hold...


In [140]:
df['review']

0       This game can suck my balls before I play it a...
1       Very fun little party game! Even better with f...
2       if you're lonely don't bother but if you're no...
3                                        fun and anoyying
4                                 when impostor is sus...
                              ...                        
1456                       Good. Been better in the past.
1457    Great game but man the community is full of 12...
1458       I like smashing skulls in with a sledge hammer
1459                                         why do i die
1460                                            lol siege
Name: review, Length: 1461, dtype: object

In [142]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.metrics import classification_report


# Text Preprocessing
vectorizer = TfidfVectorizer()
vectorizer.fit(rev_df['tokenized_sentence'])

X = vectorizer.transform(df['review'])
y = df['constructive']

# Stratified Split into Train-Test Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Compute class weights
from sklearn.utils.class_weight import compute_class_weight
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight_dict = {classes[i]: class_weights[i] for i in range(len(classes))}

# Models to Evaluate
models = {
    'Logistic Regression': LogisticRegression(class_weight=class_weight_dict, max_iter=1000),
    'Random Forest': RandomForestClassifier(class_weight=class_weight_dict, n_estimators=100),
    'SVM': SVC(class_weight=class_weight_dict, probability=True),
    'Naive Bayes': MultinomialNB(),
    #'XGBoost': XGBClassifier(scale_pos_weight=class_weight_dict[1])
}

# Cross-validation settings
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def evaluate_model(model, param_grid):
    grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    #best_model.fit(X_train, y_train)  # Ensure the best model is trained
    y_pred = best_model.predict(X_test)
    print(f"Best Model: {best_model}")
    print(classification_report(y_test, y_pred))

# Hyperparameter Grids
param_grids = {
    'Logistic Regression': {'C': [0.1, 1, 10]},
    'Random Forest': {'n_estimators': [50, 100, 200]},
    'SVM': {'C': [0.1, 1, 10]},
    'Naive Bayes': {},
    #'XGBoost': {'learning_rate': [0.01, 0.1, 0.2]} #XGBoost requires an unsupported version of sklearn that's incompatible with this environment's setup
}

# Train and Evaluate Models
for model_name, model in models.items():
    print(f"Training {model_name}...")
    #model.fit(X_train, y_train)  # Fit each model
    evaluate_model(model, param_grids[model_name])


Training Logistic Regression...
Best Model: LogisticRegression(C=10,
                   class_weight={0: 0.7934782608695652, 1: 1.3518518518518519},
                   max_iter=1000)
              precision    recall  f1-score   support

           0       0.86      0.82      0.84       185
           1       0.72      0.77      0.74       108

    accuracy                           0.80       293
   macro avg       0.79      0.80      0.79       293
weighted avg       0.81      0.80      0.80       293

Training Random Forest...
Best Model: RandomForestClassifier(class_weight={0: 0.7934782608695652,
                                     1: 1.3518518518518519})
              precision    recall  f1-score   support

           0       0.79      0.94      0.86       185
           1       0.84      0.57      0.68       108

    accuracy                           0.80       293
   macro avg       0.81      0.75      0.77       293
weighted avg       0.81      0.80      0.79       293

Trai

In [143]:
import sys
print(sys.prefix)

C:\Users\count\anaconda3\envs\cuda_test2


In [144]:
import torch
print(torch.cuda.is_available())

True


In [147]:
rev_df.head()

,recommendationid,clean_sentence,tokenized_sentence,voted_up,order,informative,heuristic,RoBERTa-Twitter_sentiment,RoBERTa-Twitter_score,VADER_sentiment,VADER_score,Sarcasm_Prob,RoBERTa-Twitter_score_standardized,VADER_score_standardized,sentence_count,tokenized_sentence_no_punct,clean_sentence_no_punct
0,24845437.0,press craft item inventory menu took like minu...,press e to craft an item while in the inventor...,1.0,24.5,1.0,"[""[('U',)]""]",TBD,0.727397,TBD,0.23165,0.287373,0.197922,0.236190,2.0,press e to craft an item while in the inventor...,press craft item inventory menu took like minu...
1,24845526.0,keep pressing mouse circle fills,what this with keep pressing mouse down until ...,0.0,1.0,1.0,"[""[('U',)]""]",TBD,0.626157,TBD,0.00000,0.133741,-0.111161,0.002472,1.0,what this with keep pressing mouse down until ...,keep pressing mouse circle fills
2,24845539.0,cons horrible proformance highend simple times...,cons horrible proformance on highend pc to sim...,0.0,6.5,1.0,"[""[('U',)]""]",TBD,0.928605,TBD,-0.80275,0.078206,0.812205,-0.807446,2.0,cons horrible proformance on highend pc to sim...,cons horrible proformance highend simple times...
3,24845671.0,preorder bonus ship gives extra inventory slot...,the preorder bonus ship only gives you 1 extra...,1.0,4.0,1.0,"[""[('U',)]""]",TBD,0.852708,TBD,0.77830,0.896714,0.580493,0.787721,1.0,the preorder bonus ship only gives you 1 extra...,preorder bonus ship gives extra inventory slot...
4,24845726.0,runs like shit crappy console port,runs like shit and its just a crappy console port,0.0,0.0,1.0,"[""[('U',)]""]",TBD,0.942896,TBD,-0.69080,0.687808,0.855833,-0.694496,1.0,runs like shit and its just a crappy console port,runs like shit crappy console port


In [152]:
selected_model = models['SVM']
selected_model.fit(X_train, y_train)  # Fit each model


evaluate_model(selected_model, param_grids['SVM'])
#temp =  selected_model.predict( vectorizer.transform(rev_df['tokenized_sentence'].head(20)))
#rev_df['constructive'] = selected_model.predict( vectorizer.transform(rev_df['tokenized_sentence']))

Best Model: SVC(C=1, class_weight={0: 0.7934782608695652, 1: 1.3518518518518519},
    probability=True)
              precision    recall  f1-score   support

           0       0.83      0.86      0.85       185
           1       0.75      0.70      0.73       108

    accuracy                           0.81       293
   macro avg       0.79      0.78      0.79       293
weighted avg       0.80      0.81      0.80       293



In [154]:
temp =  selected_model.predict_proba( vectorizer.transform(rev_df['tokenized_sentence'].head(20)))[:,1]
print(temp)

[0.84099324 0.21762012 0.20838754 0.16383369 0.21596861 0.07378267
 0.43260489 0.10555877 0.18904502 0.51893875 0.0787293  0.73790336
 0.312949   0.51873175 0.14518417 0.52138672 0.24502714 0.33560503
 0.72919992 0.08854642]


In [156]:
print(temp)

[0.84099324 0.21762012 0.20838754 0.16383369 0.21596861 0.07378267
 0.43260489 0.10555877 0.18904502 0.51893875 0.0787293  0.73790336
 0.312949   0.51873175 0.14518417 0.52138672 0.24502714 0.33560503
 0.72919992 0.08854642]


In [160]:
#rev_df['constructive'] = selected_model.predict( vectorizer.transform(rev_df['tokenized_sentence']))
rev_df['constructive'] = selected_model.predict( vectorizer.transform(rev_df['tokenized_sentence_no_punct']))

In [161]:
rev_df['constructive_probability'] = selected_model.predict_proba( vectorizer.transform(rev_df['tokenized_sentence_no_punct']))[:,1]

In [162]:
print(len(rev_df))
rev_df.tail(20)

68231


,recommendationid,clean_sentence,tokenized_sentence,voted_up,order,informative,heuristic,RoBERTa-Twitter_sentiment,RoBERTa-Twitter_score,VADER_sentiment,VADER_score,Sarcasm_Prob,RoBERTa-Twitter_score_standardized,VADER_score_standardized,sentence_count,tokenized_sentence_no_punct,clean_sentence_no_punct,constructive,constructive_probability
68211,70200694.0,wont let launch game tried everything fix spen...,it wont let me launch the game tried everythin...,0.0,0.0,1.0,"[""[('F',)]""]",TBD,0.850972,TBD,0.00000,0.235257,0.573313,0.004682,1.0,it wont let me launch the game tried everythin...,wont let launch game tried everything fix spen...,0,0.294845
68212,70219742.0,poor reviews prevented purchasing title past,poor reviews had prevented me from purchasing ...,1.0,0.0,1.0,"[""[('F',)]""]",TBD,0.825934,TBD,-0.45880,0.052437,0.496528,-0.457236,1.0,poor reviews had prevented me from purchasing ...,poor reviews prevented purchasing title past,0,0.154359
68213,70230531.0,still play serious issues,i still did play it but it had serious issues,1.0,3.0,1.0,"[""[('F',)]""]",TBD,0.849133,TBD,0.06440,0.050867,0.567673,0.069519,1.0,i still did play it but it had serious issues,still play serious issues,1,0.546336
68214,70245980.0,cause failed launch promises,cause of the failed launch promises,1.0,1.0,1.0,"[""[('F',)]""]",TBD,0.792496,TBD,-0.17790,0.854115,0.393985,-0.174427,1.0,cause of the failed launch promises,cause failed launch promises,0,0.166240
68215,70265878.0,cons multiplayer bugs multiplayer fully functi...,cons multiplayer bugs while multiplayer is ful...,1.0,17.0,1.0,"[""[('F',)]""]",TBD,0.606681,TBD,-0.17790,0.480980,-0.175844,-0.174427,1.0,cons multiplayer bugs while multiplayer is ful...,cons multiplayer bugs multiplayer fully functi...,1,0.712029
68216,70279445.0,require beefy computer though quad 250 ghz 118...,does require a beefy computer though i have a ...,1.0,3.0,1.0,"[""[('F',)]""]",TBD,0.497149,TBD,0.63690,0.049309,-0.511740,0.645910,1.0,does require a beefy computer though i have a ...,require beefy computer though quad 250 ghz 118...,0,0.260069
68217,70339207.0,give thumbs let change settings starting new game,i will give you a thumbs up when you let me ch...,0.0,0.0,1.0,"[""[('F',)]""]",TBD,0.623303,TBD,0.00000,0.260157,-0.124870,0.004682,1.0,i will give you a thumbs up when you let me ch...,give thumbs let change settings starting new game,0,0.181842
68218,70341291.0,tried land planet quit game try restart since ...,then i tried to land it on the planet and quit...,0.0,2.0,1.0,"[""[('F',)]""]",TBD,0.712746,TBD,0.00000,0.459554,0.149421,0.004682,1.0,then i tried to land it on the planet and quit...,tried land planet quit game try restart since ...,1,0.592193
68219,70344029.0,tried several hours fix bug tutorial missions ...,tried for several hours to fix bug in tutorial...,0.0,0.0,1.0,"[""[('F',)]""]",TBD,0.683752,TBD,0.49390,0.692879,0.060505,0.501938,1.0,tried for several hours to fix bug in tutorial...,tried several hours fix bug tutorial missions ...,0,0.351706
68220,70345784.0,first plan playing need high end hardware. 740...,first if you plan on playing in vr you need to...,1.0,7.4,1.0,"[""[('F',)]""]",TBD,0.797527,TBD,0.07862,0.295569,0.409415,0.083836,5.0,first if you plan on playing in vr you need to...,first plan playing need high end hardware 7400...,1,0.895261


In [163]:
rev_df.head(10)

,recommendationid,clean_sentence,tokenized_sentence,voted_up,order,informative,heuristic,RoBERTa-Twitter_sentiment,RoBERTa-Twitter_score,VADER_sentiment,VADER_score,Sarcasm_Prob,RoBERTa-Twitter_score_standardized,VADER_score_standardized,sentence_count,tokenized_sentence_no_punct,clean_sentence_no_punct,constructive,constructive_probability
0,24845437.0,press craft item inventory menu took like minu...,press e to craft an item while in the inventor...,1.0,24.5,1.0,"[""[('U',)]""]",TBD,0.727397,TBD,0.23165,0.287373,0.197922,0.236190,2.0,press e to craft an item while in the inventor...,press craft item inventory menu took like minu...,1,0.840993
1,24845526.0,keep pressing mouse circle fills,what this with keep pressing mouse down until ...,0.0,1.0,1.0,"[""[('U',)]""]",TBD,0.626157,TBD,0.00000,0.133741,-0.111161,0.002472,1.0,what this with keep pressing mouse down until ...,keep pressing mouse circle fills,0,0.217620
2,24845539.0,cons horrible proformance highend simple times...,cons horrible proformance on highend pc to sim...,0.0,6.5,1.0,"[""[('U',)]""]",TBD,0.928605,TBD,-0.80275,0.078206,0.812205,-0.807446,2.0,cons horrible proformance on highend pc to sim...,cons horrible proformance highend simple times...,0,0.208388
3,24845671.0,preorder bonus ship gives extra inventory slot...,the preorder bonus ship only gives you 1 extra...,1.0,4.0,1.0,"[""[('U',)]""]",TBD,0.852708,TBD,0.77830,0.896714,0.580493,0.787721,1.0,the preorder bonus ship only gives you 1 extra...,preorder bonus ship gives extra inventory slot...,0,0.163834
4,24845726.0,runs like shit crappy console port,runs like shit and its just a crappy console port,0.0,0.0,1.0,"[""[('U',)]""]",TBD,0.942896,TBD,-0.69080,0.687808,0.855833,-0.694496,1.0,runs like shit and its just a crappy console port,runs like shit crappy console port,0,0.215969
5,24845880.0,1110 would molest virgin planet,1110 would molest a virgin planet again,1.0,0.0,1.0,"[""[('U',)]""]",TBD,0.791999,TBD,-0.47670,0.319447,0.395149,-0.478485,1.0,1110 would molest a virgin planet again,1110 would molest virgin planet,0,0.073783
6,24845905.0,pvp aspect awful half naked man run hill demol...,the pvp aspect is awful as a half naked man ca...,0.0,4.0,1.0,"[""[('U',)]""]",TBD,0.929842,TBD,0.05160,0.504178,0.815982,0.054533,1.0,the pvp aspect is awful as a half naked man ca...,pvp aspect awful half naked man run hill demol...,0,0.432605
7,24845989.0,changing mouse sensitivity unresponsive,changing mouse sensitivity is unresponsive,0.0,1.0,1.0,"[""[('U',)]""]",TBD,0.725801,TBD,0.00000,0.294270,0.193051,0.002472,1.0,changing mouse sensitivity is unresponsive,changing mouse sensitivity unresponsive,0,0.105559
8,24846075.0,well far terrible experince. could use help,well so far a terrible experince. could use so...,0.0,1.5,1.0,"[""[('U',)]""]",TBD,0.695640,TBD,0.04425,0.400029,0.100970,0.047117,2.0,well so far a terrible experince could use som...,well far terrible experince could use help,0,0.189045
9,24846136.0,cons poorly optimized action takes longer hold...,cons very poorly optimized doing any action ta...,0.0,0.0,1.0,"[""[('U',)]""]",TBD,0.914853,TBD,0.14770,0.120171,0.770221,0.151491,1.0,cons very poorly optimized doing any action ta...,cons poorly optimized action takes longer hold...,0,0.518939


In [168]:
rev_df.to_csv('datasets/__sprint2_All_heuristics_review_sentences_remerged_with_constructive_scoring.csv', index=False)